In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Our first step is to create a "scene" on the sky, populating a chunk of sky with point sources.
# Though to start, we probably just want one point source. So we'll make a blank image, and then
# put a point source in the middle:

a0=np.zeros([301,301])
a0[150,150] = 301**2
# Note, if you wanted to make more "stars", you could do this here.


# And to make sure everything works, let's plot the image:

plt.figure(figsize=(12,12))
plt.imshow(a0)
plt.colorbar()
plt.tight_layout(); plt.show(); plt.close()

In [ ]:
# Now that we have a chunk of sky, let's simulate observing it - or more specifically, what the
# wavefront of light would look like when it gets to us.
#
# To do this, we take the inverse fourier transform (ifft2) of the scene on the sky.

a1 = np.fft.ifft2(a0) # Note, fftshift would move "zero" to the middle, in case we need to worry about phase

#And again, let's plot it. For display purposes, let's show the "amplitude" (power) using the np.abs command.

plt.figure(figsize=(12,12))
plt.imshow(np.abs(a1)**2)
plt.colorbar()
plt.tight_layout(); plt.show(); plt.close()

In [ ]:
# Now we want to define a telescope aperture for the light to pass through (or not pass through).
# So again we make a blank image, then set parts of it equal to 1 where the light should pass through.
#
# Note that lines 2-4 of this little snippet of code can be used to reference all pixels within a
# user-defined distance of a user-defined point. This is very handy for image manipulation.

m1 = np.zeros([301,301])
telradius=30
xx,yy = np.meshgrid(np.arange(301)-150,np.arange(301)-150)
r=np.hypot(xx,yy)
m1[np.where(r<telradius)] = 1

# Note that if you wanted to define a central obstruction, this is where you'd do it. 
# But if you have a coronagraph, note that it changes how it behaves and you might need to change the Lyot stop.

# Note that if you wanted to define "spiders" (the structure holding the secondary mirror), you can do it here too.


# And now let's plot our aperture model.'

plt.figure(figsize=(12,12))
plt.imshow(m1)
plt.colorbar()
plt.tight_layout(); plt.show(); plt.close()

In [ ]:
# Now that we have an incoming wavefront and an aperture model, we can multiply them to get
# the wavefront that passes through the aperture.

a1m1 = a1*m1

# And plot it.

plt.figure(figsize=(12,12))
plt.imshow(np.abs(a1m1)**2)
plt.colorbar()
plt.tight_layout(); plt.show(); plt.close()

In [ ]:
# Now we take the Fourier transform (fft2) to produce the light distribution in the focal plane.
# If this is squared, it's the "point spread function" or PSF.

a2 = np.fft.fft2(a1m1)

# And make an image of the PSF. Note that in this case, we want to change the stretch so we see the wings.
# An easy way to do this is to plot the square root, third root, or more.

plt.figure(figsize=(12,12))
plt.imshow( (np.abs(a2)**2)**0.25 )
plt.colorbar()
plt.tight_layout(); plt.show(); plt.close()

In [ ]:
# Bonus exercise - now we define a Lyot coronagraph (hard-edged opaque circle) as a model array.

m2 = np.zeros([301,301])
m2 = m2 + 1
corradius = 30
xx,yy = np.meshgrid(np.arange(301)-150,np.arange(301)-150)
r = np.hypot(xx,yy)
m2[np.where(r<corradius)] = 0

# And plot it.

plt.figure(figsize=(12,12))
plt.imshow(m2)
plt.colorbar()
plt.tight_layout(); plt.show(); plt.close()

In [ ]:
# And then we multiply the Lyot coronagraph "aperture model" and the wavefront at the focal plane.

a2m2 = a2*m2

# And plot it.

plt.figure(figsize=(12,12))
plt.imshow( (np.abs(a2m2)**2)**0.25 )
plt.colorbar()
plt.tight_layout(); plt.show(); plt.close()

In [ ]:
# But we can't stop just yet. We now need a collimator to bring the light back to being
# "collimated", with focus at infinity, so we can do more manipulation.

a3=np.fft.ifft2(a2m2)

# And plot it.

plt.figure(figsize=(12,12))
plt.imshow(np.abs(a3)**2)
plt.colorbar()
plt.tight_layout(); plt.show(); plt.close()

In [ ]:
# All the residual light has now diffracted around the coronagraph, and most of the intensity is
# at a couple of distinct radii. So we want to make a "pupil stop" or "Lyot stop" model.

m3 = np.zeros([301,301])
m3 = m3 + 1
xx,yy = np.meshgrid(np.arange(301)-150,np.arange(301)-150)
r = np.hypot(xx,yy)
lyot1=25
lyot2=33
m3[np.where((r>lyot1) & (r<lyot2))] = 0

# And plot it.

plt.figure(figsize=(12,12))
plt.imshow(m3)
plt.colorbar()
plt.tight_layout(); plt.show(); plt.close()

In [ ]:
# As at the focal plane, we now want to multiply the incoming wavefront and the Lyot stop model to show
# what passes onward along the optical path.

a3m3 = a3*m3

# And plot it.

plt.figure(figsize=(12,12))
plt.imshow(np.abs(a3m3)**2)
plt.colorbar()
plt.tight_layout(); plt.show(); plt.close()

In [ ]:
# Finally, we Fourier transform that wavefront one more time, to show what it looks like in the final focal plane
# with the effects of the coronagraph and the Lyot stop.

a4 = np.fft.fft2(a3m3)

# And we plot it. We should see that not only is light blocked in the center, but the Airy rings that are
# nominally "beyond" the coronagraph are also suppressed. And thus we see the magical effect of treating
# light like a wave, and not just particles.

plt.figure(figsize=(12,12))
plt.imshow((np.abs(a4)**0.5))
plt.colorbar()
plt.tight_layout(); plt.show(); plt.close()

In [ ]:
# Note, there are a number of things we can do to make this a more realistic exercise.
# If we have time, we'll investigate implementing some:
# 
# 1) Add central obstruction from the secondary mirror blocking light to primary mirror.
# 2) Add spiders that support the secondary mirror in place.
# 3) Add a faint companion.
# 4) Add phase noise, produce speckles?
# 5) Add tons of phase noise, make a seeing-limited image.